In [1]:
!pip install pandasql

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

## Importing pandasql 
import pandasql as psql
import matplotlib.pyplot as plt

In [3]:
JobData = pd.read_csv('indeed_job_dataset.csv')

# 1.STATE WISE COUNT OF JOB OPENINGS

In [4]:
State_Open = psql.sqldf("SELECT DISTINCT Location as STATE, count(*) AS No_Of_Openings, ((Count(*)* 100 / (Select Count(*) From JobData where Location <>'NONE')) || '%') AS Percentage\
                                from JobData\
                                where Location <>'NONE'\
                                GROUP BY Location\
                                 order by No_Of_Openings DESC LIMIT 5" )
State_Open

,STATE,No_Of_Openings,Percentage
0,CA,1395,25%
1,NY,601,11%
2,VA,334,6%
3,TX,329,6%
4,MA,271,4%


# 2.LOCATION WISE MOST SKILL DEMANDING JOBS

In [10]:
job_date_skill = psql.sqldf("SELECT distinct  Skill,Location, Company, Date_Since_Posted,No_of_Skills	, count(*) AS No_of_unfilled_Jobs\
                               FROM JobData where Date_Since_Posted = 30 \
                                GROUP BY Skill, Company,Location\
                                ORDER BY No_of_unfilled_Jobs DESC LIMIT 10")

job_date_skill

,Skill,Location,Company,Date_Since_Posted,No_of_Skills,No_of_unfilled_Jobs
0,"['R', 'SAS', 'Python', 'SQL', 'Tableau']",CA,All-In Analytics,30.0,5,10
1,"['TensorFlow', 'Machine Learning', 'TS/SCI Cle...",MD,Booz Allen Hamilton,30.0,12,10
2,"['TensorFlow', 'Hive', 'Machine Learning', 'Ha...",NY,JPMorgan Chase,30.0,13,6
3,"['Pig', 'Hive', 'Machine Learning', 'Experimen...",NY,McGraw Hill Financial,30.0,20,5
4,None,CA,Genentech,30.0,0,4
5,"['Analysis Skills', 'Machine Learning', 'R', '...",TX,Lockheed Martin Corporation,30.0,5,4
6,['Data Analysis'],NC,Capgemini,30.0,1,4
7,"['Data Management', 'Hadoop', 'Kafka', 'HBase'...",IL,Capgemini,30.0,8,4
8,"['Hive', 'Machine Learning', 'Hadoop', 'Kafka'...",NY,Citi,30.0,7,4
9,"['Java', 'Natural Language Processing', 'Machi...",CA,Walmart,30.0,5,4


# 3.MOST PREFERRED STATE/LOCATION TO WORK AS PER RATINGS

In [11]:
job_satisfaction = psql.sqldf("SELECT  distinct Location,Queried_Salary	, No_of_Reviews,avg(No_Of_stars) AS Avg_Rating,\
                                ((No_of_stars*No_of_Reviews)/( SELECT sum(No_of_Reviews) FROM JobData))AS Weighted_Rating\
                                from JobData  WHERE No_of_Reviews > 4000\
                                GROUP BY Location \
                                ORDER BY Weighted_Rating DESC   LIMIT 5")
job_satisfaction

,Location,Queried_Salary,No_of_Reviews,Avg_Rating,Weighted_Rating
0,TX,80000-99999,157456.0,3.940698,0.027662
1,AR,80000-99999,157456.0,3.630000,0.027662
2,IA,100000-119999,29963.0,3.800000,0.005556
3,USA,80000-99999,23472.0,3.947826,0.004467
4,OH,100000-119999,21159.0,3.983871,0.004027


# 4.MOST POPULAR SKILLS LOCATION WISE

In [17]:
pq = psql.sqldf("SELECT    distinct Location, sum(python) As Python_Jobs,\
                                   sum(sql) As SQL,  sum(r) AS R, sum(hadoop) AS Hadoop,\
                                   sum(tableau) AS Tableau, sum(sas) AS SAS, sum(spark) AS Spark, sum(Java) As JAVA, sum(Others) As Others\
                                from JobData\
                                where Location  not like '%None%'\
                                 GROUP BY Location\
                                   LIMIT 5       ")           
pq                     

,Location,Python_Jobs,SQL,R,Hadoop,Tableau,SAS,Spark,JAVA,Others
0,AL,10,9,9,4,2,2,2,4,16
1,AR,11,16,17,15,2,14,9,5,26
2,AZ,34,29,32,19,15,14,13,15,49
3,CA,933,787,531,454,289,170,419,375,1216
4,CO,74,68,49,39,19,4,30,40,103


# 5.MOST DIFFICULT JOBS TO FILL LOCATION AND COMPANY WISE

In [18]:
job_date_cmp = psql.sqldf("SELECT distinct Location,Company , Company_Employees, Date_Since_Posted, count(*) AS No_of_unfilled_Jobs\
                               FROM JobData where Date_Since_Posted = 30 \
                                GROUP BY Company,Location\
                                ORDER BY No_of_unfilled_Jobs DESC LIMIT 10")

job_date_cmp

,Location,Company,Company_Employees,Date_Since_Posted,No_of_unfilled_Jobs
0,CA,Harnham,"Less than 10,000",30.0,48
1,CA,Walmart,"10,000+",30.0,29
2,VA,Booz Allen Hamilton,"10,000+",30.0,25
3,CA,Walmart eCommerce,"Less than 10,000",30.0,23
4,WA,Microsoft,"10,000+",30.0,21
5,CA,Uber,"10,000+",30.0,21
6,MD,Booz Allen Hamilton,"10,000+",30.0,20
7,DC,Booz Allen Hamilton,"10,000+",30.0,19
8,VA,Capital One,"10,000+",30.0,19
9,NY,Harnham,"Less than 10,000",30.0,16


# 6.TOP 5 DATA ANALYST ROLES COMPANY AND LOCATION WISE

In [20]:
DAT = psql.sqldf("select Location,Company,Job_Title, count(Job_Title) as no_of_openings from JobData where Job_Type = 'data_analyst' group by Location,Company,Job_Title order by count(Job_Title) desc limit 5")
DAT

,Location,Company,Job_Title,no_of_openings
0,NC,Capgemini,Data Analyst,12
1,CA,University of California San Francisco,Research Data Analyst,6
2,NY,Capgemini,Data Analyst,6
3,MO,Centene,Data Analyst II,5
4,MO,Centene,Data Analyst III,5


# 7.HIGHEST AVERAGE SALARY ACROSS COMPANIES

In [26]:
AS = psql.sqldf("select Company, avg(Queried_Salary) as Average_Salary from JobData where Queried_Salary is not null AND Company not like '%None%' group by Company order by avg(Queried_Salary) desc limit 10")
AS

,Company,Average_Salary
0,zesty.ai,140000.0
1,thredUP Inc,140000.0
2,stanleyreid,140000.0
3,lululemon athletica,140000.0
4,iRobot Corporation,140000.0
5,iHerb,140000.0
6,drawbridge,140000.0
7,Zot Inc.,140000.0
8,ZocDoc,140000.0
9,Zazzle,140000.0


# 8.JOB ROLE IN DEMAND COMPANY WISE

In [27]:
CJT = psql.sqldf("select distinct Job_Type,Company,count(*) as open_positions from JobData group by Company,Job_Type order by count(*) desc limit 10")
CJT.head()

,Job_Type,Company,open_positions
0,data_scientist,Booz Allen Hamilton,114
1,data_engineer,None,75
2,data_analyst,Capgemini,55
3,data_scientist,KPMG LLP,51
4,data_scientist,Walmart,51


# 9.COMPANIES WITH THE HIGHEST OPENINGS THAT HAVENT BEEN FILLED IN A MONTH

In [30]:
job_count = psql.sqldf("SELECT Company,count(Job_Title) , Date_Since_Posted\
                               FROM JobData WHERE Date_Since_Posted = 30\
                               GROUP BY Company\
                                ORDER BY count(Job_Title) DESC limit 10")

job_count

,Company,count(Job_Title),Date_Since_Posted
0,Booz Allen Hamilton,78,30.0
1,Harnham,69,30.0
2,KPMG LLP,47,30.0
3,Walmart,38,30.0
4,Capgemini,33,30.0
5,Capital One,29,30.0
6,Allstate,28,30.0
7,Uber,25,30.0
8,Microsoft,25,30.0
9,Walmart eCommerce,23,30.0


# 10.DATA ANALYST HUB

In [43]:
job_ds_loc = psql.sqldf("SELECT     Location, count(Job_Title) As No_Of_DataAnalyst_Jobs,\
                                  ((Count(Job_Title)* 100 / (SELECT     count(Job_Title) As No_Of_DataAnalyst_Jobs from JobData WHERE Job_Title like '%data_analyst%')) \
                                  || '%') AS PCT\
                                from JobData\
                                 WHERE Job_Title like '%data_analyst%'\
                                  GROUP BY Location\
                                  ORDER BY No_Of_DataAnalyst_Jobs DESC\
                                  LIMIT 10       ")           
                                

job_ds_loc

,Location,No_Of_DataAnalyst_Jobs,PCT
0,CA,316,19%
1,NY,190,11%
2,TX,110,6%
3,VA,76,4%
4,MA,76,4%
5,IL,63,3%
6,MD,54,3%
7,WA,51,3%
8,GA,51,3%
9,PA,49,3%


# 11.COMPANY WISE MOST DEMANDED SKILLS

In [40]:
PLJT = psql.sqldf("select Company, sum(java) as Java, sum(python) as Python, sum(sas) as SAS, sum(r) as R \
                          from JobData WHERE Company NOT LIKE '%None%' group by Company order by sum(python)  DESC LIMIT 10 ")
PLJT

,Company,Java,Python,SAS,R
0,Booz Allen Hamilton,54,91,24,81
1,Harnham,17,62,5,29
2,Facebook,20,55,27,36
3,KPMG LLP,2,54,0,12
4,Walmart,22,39,15,33
5,Capital One,39,39,0,6
6,Allstate,31,33,10,31
7,Jobspring Partners,15,29,3,10
8,Uber,2,28,0,24
9,IBM,14,28,12,23


# 12.MOST PREFERRED COMPANIES TO WORK AS PER RATINGS

In [42]:

job_satisfaction = psql.sqldf("SELECT  distinct Company,Queried_Salary	, No_of_Reviews,avg(No_Of_stars) AS Avg_Rating,\
                                ((No_of_stars*No_of_Reviews)/( SELECT sum(No_of_Reviews) FROM JobData))AS Weighted_Rating\
                                from JobData  WHERE No_of_Reviews > 4000\
                                GROUP BY Company \
                                ORDER BY Weighted_Rating DESC   LIMIT 10")
job_satisfaction

,Company,Queried_Salary,No_of_Reviews,Avg_Rating,Weighted_Rating
0,Walmart,80000-99999,157456.0,3.6,0.027662
1,The Home Depot,100000-119999,41290.0,3.8,0.007657
2,SUBWAY,<80000,33082.0,3.7,0.005973
3,AT&T,80000-99999,31970.0,3.8,0.005928
4,Starbucks,80000-99999,27858.0,4.1,0.005574
5,Wells Fargo,100000-119999,29963.0,3.8,0.005556
6,Lowe's,120000-139999,29206.0,3.7,0.005273
7,UPS,100000-119999,27660.0,3.9,0.005264
8,IBM,80000-99999,23472.0,3.9,0.004467
9,Kroger,<80000,24248.0,3.6,0.004260


# 13.COMPANIES WITH THE LEAST NUMBER OF OPENINGS

In [48]:
job_cmp = psql.sqldf("SELECT  distinct Company, count(*) As No_Of_Jobs, ((Count(*)* 100 / (Select Count(*) From JobData)) || '%') AS PCT\
                                from JobData \
                                GROUP BY Company HAVING Company <>'NONE'\
                                ORDER BY No_Of_Jobs asc LIMIT 5")
job_cmp.head()

,Company,No_Of_Jobs,PCT
0,14 West,1,0%
1,1st Solution USA,1,0%
2,2K Games,1,0%
3,2U,1,0%
4,3Q Digital,1,0%


# 14.INDUSTRY WITH THE LEAST NUMBER OF OPENINGS

In [50]:
job_ind = psql.sqldf("SELECT  distinct Company_Industry, count(*) As No_Of_Jobs, ((Count(*)* 100 / (Select Count(*) From JobData)) || '%') AS PCT\
                                from JobData \
                                GROUP BY Company_Industry HAVING Company_Industry <>'NONE'\
                                ORDER BY No_Of_Jobs asc LIMIT 5")
job_ind

,Company_Industry,No_Of_Jobs,PCT
0,Food and BeveragesConsulting and Business Serv...,1,0%
1,Industrial ManufacturingAgriculture and Extrac...,1,0%
2,Industrial ManufacturingConstruction,1,0%
3,Industrial ManufacturingConsumer Goods and Ser...,2,0%
4,Real EstateReal Estate,2,0%


# 15.LOCATION WISE HIGHEST PAYING JOB ROLES

In [55]:
AS = psql.sqldf("select Location,Job_Type, avg(Queried_Salary) as Average_Salary from JobData where Queried_Salary is not null AND Location NOT LIKE '%None%' group by Job_Type,Location ORDER BY avg(Queried_Salary) desc LIMIT 5")
AS

,Location,Job_Type,Average_Salary
0,REMOTE,data_scientist,116000.000000
1,MO,data_engineer,114285.714286
2,NH,data_scientist,113333.333333
3,VA,data_engineer,113055.555556
4,USA,data_scientist,112820.512821
